In [13]:
import requests
import datetime
import os
import csv
import pandas as pd
# from os import environ
from sqlalchemy import MetaData, create_engine, asc
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

import psycopg2
from psycopg2 import Error

from flask import Flask, jsonify, request
from flask_cors import cross_origin, CORS
from models.models import Base

from models.models import ExpensesRaw
# from endpointClasses.resources import Resources
#
from sqlalchemy import Column, Integer, BigInteger, String, Text, DateTime, \
    Float, Boolean, func, ForeignKeyConstraint, Index, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from sqlalchemy.ext.hybrid import hybrid_property, hybrid_method
from datetime import datetime, timedelta, timezone
from geoalchemy2 import Geography, Geometry


In [2]:
### Setup the application
app = Flask(__name__)

# Wrap CORS around the app so that the server does not block machine to machine
# or browser based requests
CORS(app)

In [3]:
# Engine below for Google Cloud PostgreSQL access.
engine = create_engine('postgresql+psycopg2://postgres:5413CrossFit2018@34'
                       '.70.40.80/transgov')

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

In [4]:
col_names = ["Ministry", "Position", "Name", "Type", "Category", "Date", "Amount",
         "Description", "Receipt 1", "Receipt 2", "Receipt 3"]

In [16]:
# download  csv file
# Only do this once per day
print('Downloading .....')
df = pd.read_csv(r'https://expenses.alberta.ca/DownloadData.aspx?type=csv'
                   r'&d=IsVE/OcdpNZJ5rBbvji3qw', names=col_names,
                     low_memory=False, parse_dates=['Date'])

# Saving the dataframe
print('Saving to CSV ...')
df.to_csv('expenses.csv')
print('done...')

Saving to CSV ...
done...


In [38]:
print('Reading from file...')
df=pd.read_csv('expenses.csv', names=col_names, low_memory=False, skiprows=2)
print('Done...')

Reading from file...
Done...


In [39]:
# Process the dataframe
# Note that after the very first attempt at this we need to test for whether the record already exist and then update the
# date last found
length = len(df)
ctr = 0
for index, row in df.iterrows():
    
    newdate = df._get_value(index, 'Date')
    newdate = datetime.strptime(newdate, '%m/%d/%Y')
#     print('New Date', newdate)
    newAmount = float(df._get_value(index, 'Amount').replace('$', '').replace(',',''))
#     print('New Amount', newAmount)
    
    expense = ExpensesRaw(
        ministry=df._get_value(index, 'Ministry'),
        position=df._get_value(index, 'Position'),
        name=df._get_value(index, 'Name'),
        type=df._get_value(index, 'Type'),
        category=df._get_value(index, 'Category'),
        expense_date=newdate,
        amount=newAmount,
        description=df._get_value(index, 'Description'),
        receipt1=df._get_value(index, 'Receipt 1'),
        receipt2=df._get_value(index, 'Receipt 2'),
        receipt3=df._get_value(index, 'Receipt 3'),
        date_last_found=datetime.now(),
        created_at=datetime.now(),
        updated_at=datetime.now(),
    )
    
    if ctr % 100 == 0:
        print('Count: %s of %s ' %(ctr, length))
    
    ctr = ctr + 1
    
    try:
        session.add(expense)
        session.commit()
    except SQLAlchemyError as e:
#         error = str(e.__dict__['orig'])
        session.rollback()
        print('Could not save the new expense %s',
              expense)
#         print(error)
   

Count: 0 of 536436 
Could not save the new expense %s ExpensesRaw(id='None' , ministry='Agriculture and Rural Development' , position='Executive Director' , name='Munroe,Clifford' , type='Travel' , expense_date='2012-10-10 00:00:00' , amount='20.75' , description='Counter Terrorism Seminar: Dinner allowance' , receipt1='nan' , receipt2='nan' , receipt3='nan' , date_last_found='2021-01-28 09:54:28.737624' , created_at='2021-01-28 09:54:28.737624' , updated_at='2021-01-28 09:54:28.737624')
Count: 100 of 536436 
Count: 200 of 536436 
Count: 300 of 536436 
Count: 400 of 536436 
Count: 500 of 536436 
Count: 600 of 536436 
Count: 700 of 536436 
Count: 800 of 536436 
Count: 900 of 536436 
Count: 1000 of 536436 
Count: 1100 of 536436 


KeyboardInterrupt: 

In [8]:
print(df.head())

                            Ministry            Position             Name  \
1  Agriculture and Rural Development  Executive Director  Munroe,Clifford   
2  Agriculture and Rural Development  Executive Director  Munroe,Clifford   
3  Agriculture and Rural Development  Executive Director  Munroe,Clifford   
4  Agriculture and Rural Development  Executive Director  Munroe,Clifford   
5  Agriculture and Rural Development  Executive Director  Munroe,Clifford   

     Type     Category        Date   Amount  \
1  Travel        Meals  10/10/2012   $20.75   
2  Travel      Mileage  10/10/2012  $333.81   
3  Travel  Other Costs  10/12/2012    $7.35   
4  Travel  Other Costs  10/11/2012    $7.35   
5  Travel        Meals  10/12/2012   $11.60   

                                         Description Receipt 1 Receipt 2  \
1        Counter Terrorism Seminar: Dinner allowance       NaN       NaN   
2  Counter Terrorism Seminar: Edmonton/Calgary re...       NaN       NaN   
3                Counter T